In [ ]:
import numpy as np
import scipy as sc
import numpy
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import pyscf.lib
from pyscf.data import nist
import qml
from ase import units


def add_qmmm(calc, deltaZ, includeonly, mol):
    """
    modify hamiltonian such that Z = Z(lambda)
    """
    mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]*units.Bohr, deltaZ) # add charge dZ at position of nuclei

    def energy_nuc(self):
        """
        calculate correct nuclear charge for modified system
        """
        q = mol.atom_charges().astype(np.float)
        q[includeonly] += deltaZ
        return mol.energy_nuc(q)

    mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

    return mf

def calculate_alchpot(dm1_ao, includeonly, mol):
    # Electronic EPN from electron density
    alch_pots = []
    for site in includeonly:
        mol.set_rinv_orig_(mol.atom_coords()[site])
        alch_pots.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
    return(-np.array(alch_pots))

def calculate_average_alchpots(alchpots, lam_vals, intg_meth):
    num_alchpots = len(alchpots[0])
    av_alchpots = []
    for i in range(num_alchpots):
        if intg_meth == 'trapz':
            av_alchpot_i = np.trapz(alchpots[:,i], lam_vals)
        av_alchpots.append(av_alchpot_i)
    return(np.array(av_alchpots))

def get_num_elec(lam_val, total_num_elecs):
    """
    calculate number of electrons for a given lambda value and total number of electrons
    """
    if int(lam_val*total_num_elecs)%2 == 0:
        num_elec = int(lam_val*total_num_elecs)
    else:
        num_elec = int(lam_val*total_num_elecs) + 1
    return(num_elec)

def make_apdft_calc(deltaZ, includeonly, mol, method = "HF"):
    """
    SCF calculation for fractional charges defined in deltaZ
    returns the density matrix and the total energy
    """
    
    if method not in ["CCSD", "HF"]:
        raise NotImplementedError("Method %s not supported." % method)

    if method == "HF":
        calc = add_qmmm(pyscf.scf.RHF(mol), deltaZ, includeonly, mol)
        hfe = calc.kernel(verbose=0)
        dm1_ao = calc.make_rdm1()
        total_energy = calc.e_tot
    if method == "CCSD":
        calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
        hfe = calc.kernel(verbose=0)
        mycc = pyscf.cc.CCSD(calc).run()
        dm1 = mycc.make_rdm1()
        dm1_ao = np.einsum("pi,ij,qj->pq", calc.mo_coeff, dm1, calc.mo_coeff.conj())
        total_energy = mycc.e_tot
    return(dm1_ao, total_energy)

def prepare_input(coords, nuc_charges, lam_val_desired, basis = 'def2-qzvp', num_elec = None):
    num_elec = get_num_elec(lam_val_desired, nuc_charges.sum())
    lam_val = num_elec/nuc_charges.sum()
    
    mol = pyscf.gto.Mole()
    for ch, coords_atom in zip(nuc_charges, coords):
        mol.atom.append([ch, coords_atom])
    mol.basis = basis
    mol.charge = nuc_charges.sum() - num_elec
    mol.build()
    # dZ vector to generate systems for lambda != 1
    deltaZ = -nuc_charges*(1-lam_val)
    includeonly = np.arange(len(mol.atom_coords()))
    
    return(deltaZ, includeonly, mol)
    


#### Test vac $\rightarrow$ N$_2$

In [1]:
# # import everything from alchemy_tools_pyscf

# import sys
# sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
# import alchemy_tools_pyscf as atp
# import numpy as np

# lam_vals_desired = np.arange(2, 16, 2)/14
# coords = np.array([[0, 0, 0], [0, 0, 1.5]])
# nuclear_charges = np.array([7, 7])
# basis = 'def2-qzvp'
# lam_vals = []
# alchemical_potentials = []
# for lam_val_desired in lam_vals_desired:
#     num_elec = atp.get_num_elec(lam_val_desired, nuclear_charges.sum())
#     lam_vals.append(num_elec/nuclear_charges.sum())
#     print('preparing input')
#     deltaZ, includeonly, mol = atp.prepare_input(coords, nuclear_charges, num_elec, basis)
#     print('Doing SCF calculation')
#     dm, e_tot = atp.make_apdft_calc(deltaZ, includeonly, mol, method = "HF")
#     print('Calculating alchemical potentials')
#     alchpots_lambda = atp.calculate_alchpot(dm, includeonly, mol)

#     alchemical_potentials.append(alchpots_lambda)

preparing input
Doing SCF calculation


Overwritten attributes  energy_nuc  of <class 'pyscf.qmmm.itrf.qmmm_for_scf.<locals>.QMMM'>
/home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf/gto/mole.py:1047: UserWarning: Function mol.dumps drops attribute charge because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -1.00230083587087
Calculating alchemical potentials
preparing input
Doing SCF calculation
converged SCF energy = -5.70322059301431
Calculating alchemical potentials
preparing input
Doing SCF calculation
converged SCF energy = -14.6541461549333
Calculating alchemical potentials
preparing input
Doing SCF calculation
converged SCF energy = -28.9681025684945
Calculating alchemical potentials
preparing input
Doing SCF calculation
converged SCF energy = -48.8916211505035
Calculating alchemical potentials
preparing input
Doing SCF calculation
converged SCF energy = -75.3453051237298
Calculating alchemical potentials
preparing input
Doing SCF calculation
converged SCF energy = -108.711062429174
Calculating alchemical potentials


In [ ]:
lam_vals = np.arange(2, 16, 2)/14
coords = np.array([[0, 0, 0], [0, 0, 1.5]])
nuclear_charges = np.array([7, 7])
basis = 'def2-qzvp'
alchemical_potentials = []
for lam in lam_vals:
    print('preparing input')
    deltaZ, includeonly, mol = prepare_input(coords, nuclear_charges, lam, basis)
    print('Doing SCF calculation')
    dm, e_tot = make_apdft_calc(deltaZ, includeonly, mol, method = "HF")
    print('Calculating alchemical potentials')
    alchpots_lambda = calculate_alchpot(dm, includeonly, mol)

    alchemical_potentials.append(alchpots_lambda)

In [2]:
# alchemical potentials from explore_subpaths_N2.ipynb
reference_vals = np.array([[ -1.29426202,  -1.29426202],
       [ -4.09848617,  -4.09848618],
       [ -6.898463  ,  -6.898463  ],
       [ -9.87442425,  -9.87442425],
       [-13.07290408, -13.07290409],
       [-16.71126325, -16.71126261],
       [-20.63771543, -20.63771543]])

In [4]:
alchemical_potentials

[array([-1.29426202, -1.29426202]),
 array([-4.09848617, -4.09848618]),
 array([-6.898463, -6.898463]),
 array([-9.87442425, -9.87442425]),
 array([-13.07290407, -13.07290409]),
 array([-16.71126339, -16.71126248]),
 array([-20.63771543, -20.63771543])]

In [3]:
alchemical_potentials - reference_vals

array([[ 2.70037570e-09,  1.55493840e-10],
       [-3.92197474e-09,  4.98233899e-09],
       [ 4.87577978e-09,  4.03791667e-09],
       [ 3.90756938e-09,  3.23041682e-09],
       [ 5.63658631e-09, -1.48362389e-09],
       [-1.38886431e-07,  1.33181377e-07],
       [-4.30016911e-10, -4.16626733e-10]])

#### Test dsgdb9nsd_001212

In [ ]:
lam_vals = np.array([22, 30, 38, 44, 52])/52
basis = 'def2-tzvp'
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
alchemical_potentials = []


for lam in lam_vals:
    print('preparing input')
    deltaZ, includeonly, mol = prepare_input(com.coordinates, com.nuclear_charges, lam, basis)
    print('Doing SCF calculation')
    dm, e_tot = make_apdft_calc(deltaZ, includeonly, mol, method = "HF")
    print('Calculating alchemical potentials')
    alchpots_lambda = calculate_alchpot(dm, includeonly, mol)

    alchemical_potentials.append(alchpots_lambda)

In [ ]:
# get also all eigenvalues and occupation numbers
# make restart file

# save data in adequate way
# calculate average alchemical potential and store it

In [ ]:
alchemical_potentials

In [ ]:
# load reference data from different_resevoir_pyscf.ipynb
import utils_qm as ut
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')

base = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/'
alchs = ut.load_obj(base+'alch_pots')
alch_pots_reference = np.array(alchs)
# alch_pots = np.concatenate((np.array([np.zeros(12)]), alch_pots))
# homos = ut.load_obj(base+'e_homo')
# homos.insert(0,0.0)
# homos = np.array(homos)
# lumos = ut.load_obj(base+'e_lumo')
# lumos.insert(0,0.0)
# lumos = np.array(lumos)
lam_vals = np.array([0, 22, 30, 38, 44, 52])/52

In [ ]:
# reference alchpots are still positive
alch_pots_reference + alchemical_potentials

results can reproduced for N2 and dsgdb9nsd_001212

In [ ]:
fig, ax = plt.subplots(1,1)
color_dict = {1.0:'tab:blue', 6.0:'tab:green', 7.0:'tab:orange', 8.0:'tab:red'}
for i in range(len(com.nuclear_charges)):
    
    ax.plot(lam_vals, alch_pots[:,i], '-o', label = com.nuclear_charges[i], color=color_dict[com.nuclear_charges[i]])
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

In [ ]:
fig, ax = plt.subplots(1,1)
# color_dict = {1.0:'tab:blue', 6.0:'tab:green', 7.0:'tab:orange', 8.0:'tab:red'}
for i in [0,1,4,5]:
    ax.plot(lam_vals, alch_pots[:,i], '-o', label = com.nuclear_charges[i])
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)